## make_table 사용자함수 만들기

In [0]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs


def make_table(dart_data, idx, colnames, code):
    #컬럼필터링 & Transpos
    temp_data = dart_data.iloc[:,idx]
    temp_data.columns = colnames
    temp_data_T = temp_data.T

    # 종목컬럼추가
    temp_data_T.columns = temp_data_T.iloc[0]
    temp_data_T = temp_data_T.drop("type", axis=0)
    temp_data_T['code'] =  [code]*len(temp_data_T)
    col_1 = temp_data_T.columns.tolist()
    col_2 = col_1[-1:] + col_1[:-1]
    temp_data_T = temp_data_T[col_2]
    temp_data_T
    temp_data_T = temp_data_T.reset_index()
    temp_data_T = temp_data_T.rename(columns={'index' : 'DATE'})

    # 임시 프린트
    return temp_data_T

## 재무제표 가져오기

In [39]:
# 종목코드
code = "005930"
# 크롤링 주소
url = "https://finance.naver.com/item/main.nhn?code=" + code
web_content = requests.get(url)
# soup=bs(web_content.text,'html.parser')
# finance_html = soup.select('div.section.cop_analysis div.sub_section')

# pandas로 테이블 읽기
data = pd.read_html(web_content.text)
# 세번째 테이블이 재무제표
target_data = data[3]


colName = target_data.columns.values
print(colName)
yearIdx = [0]
yearName = ['type']
quaterIdx = [0]
quaterName = ['type']

for i in range(0,len(colName)): 
    if colName[i][0] == '최근 연간 실적' :
        if '(E)' not in colName[i][1]:
            yearIdx.append(i)
            yearName.append(colName[i][1])

    elif colName[i][0] == '최근 분기 실적' :
        if '(E)' not in colName[i][1]:
            quaterIdx.append(i)
            quaterName.append(colName[i][1])
            
print(yearIdx)
print(quaterIdx)
print(yearName)
print(quaterName)

[('주요재무정보', '주요재무정보', '주요재무정보') ('최근 연간 실적', '2016.12', 'IFRS연결')
 ('최근 연간 실적', '2017.12', 'IFRS연결') ('최근 연간 실적', '2018.12', 'IFRS연결')
 ('최근 연간 실적', '2019.12(E)', 'IFRS연결') ('최근 분기 실적', '2018.03', 'IFRS연결')
 ('최근 분기 실적', '2018.06', 'IFRS연결') ('최근 분기 실적', '2018.09', 'IFRS연결')
 ('최근 분기 실적', '2018.12', 'IFRS연결') ('최근 분기 실적', '2019.03', 'IFRS연결')
 ('최근 분기 실적', '2019.06(E)', 'IFRS연결')]
[0, 1, 2, 3]
[0, 5, 6, 7, 8, 9]
['type', '2016.12', '2017.12', '2018.12']
['type', '2018.03', '2018.06', '2018.09', '2018.12', '2019.03']


## 연도별 데이터 가져오기

In [40]:
year_data = make_table(target_data, yearIdx,yearName, code  )
year_data

type     DATE    code          매출액    영업이익  ... BPS(원) 주당배당금(원) 시가배당률(%) 배당성향(%)
0     2016.12  005930  2.01867e+06  292407  ...  26636      570     1.58   17.81
1     2017.12  005930  2.39575e+06  536450  ...  30427      850     1.67   14.09
2     2018.12  005930  2.43771e+06  588867  ...  35342     1416     3.66   21.92

[3 rows x 16 columns]


type,DATE,code,매출액,영업이익,당기순이익,영업이익률,순이익률,ROE(지배주주),부채비율,당좌비율,유보율,EPS(원),BPS(원),주당배당금(원),시가배당률(%),배당성향(%)
0,2016.12,005930,2.01867e+06,292407,227261,14.49,11.26,12.48,35.87,223.46,22004.1,2735,26636,570,1.58,17.81
1,2017.12,005930,2.39575e+06,536450,421867,22.39,17.61,21.01,40.68,181.61,24536.1,5421,30427,850,1.67,14.09
2,2018.12,005930,2.43771e+06,588867,443449,24.16,18.19,19.63,36.97,204.12,27531.9,6024,35342,1416,3.66,21.92


## 분기별 데이터 가져오기

In [41]:
quater_data = make_table(target_data, quaterIdx,quaterName,code )
quater_data

type     DATE    code     매출액    영업이익  ... BPS(원) 주당배당금(원) 시가배당률(%) 배당성향(%)
0     2018.03  005930  605637  156422  ...  31782      NaN      NaN     NaN
1     2018.06  005930  584827  148690  ...  33223      NaN      NaN     NaN
2     2018.09  005930  654600  175749  ...  34519      NaN      NaN     NaN
3     2018.12  005930  592650  108006  ...  35342      NaN      NaN     NaN
4     2019.03  005930  523855   62333  ...  36142      NaN      NaN     NaN

[5 rows x 16 columns]


type,DATE,code,매출액,영업이익,당기순이익,영업이익률,순이익률,ROE(지배주주),부채비율,당좌비율,유보율,EPS(원),BPS(원),주당배당금(원),시가배당률(%),배당성향(%)
0,2018.03,005930,605637,156422,116885,25.83,19.3,22.79,39.96,188.1,25279.8,1583,31782,NaN,NaN,NaN
1,2018.06,005930,584827,148690,110434,25.42,18.88,21.77,36.7,197.58,26235.7,1500,33223,NaN,NaN,NaN
2,2018.09,005930,654600,175749,131507,26.85,20.09,21.73,39.28,198.16,27412.6,1771,34519,NaN,NaN,NaN
3,2018.12,005930,592650,108006,84622,18.22,14.28,19.63,36.97,204.12,27531.9,1163,35342,NaN,NaN,NaN
4,2019.03,005930,523855,62333,50436,11.9,9.63,16.21,36.28,209.25,27833,752,36142,NaN,NaN,NaN


## 투자정보 가져오기
* encparam 값 추출

In [48]:
from pandas import DataFrame
import json

url2 = "https://navercomp.wisereport.co.kr/v2/company/c1040001.aspx?cmp_cd=" + code +"&cn=" 
# url2 = "https://companyinfo.stock.naver.com/v1/company/c1040001.aspx?cmp_cd=" + code + "&cn="
temp = requests.get(url2).text

data_to_parse = bs(temp)
lines_to_parse = []
encparam = ''

d = data_to_parse.findAll(text=True)
for line in d:
    for line2 in line.strip().split('\n'):
        if line2:
            lines_to_parse.append(line2)

for i in lines_to_parse:
    if(i.find('encparam') > 0):
        encparam = i
        break
        
encparam = encparam.split("'")[1]
print(encparam)

cW83cWV5RkJHTEs2TmxjRnpaSjhDQT09


In [50]:
url22 = "https://companyinfo.stock.naver.com/v1/company/c1040001.aspx?cmp_cd=" + code + "&cn="
temp = requests.get(url22).text

SSLError: ignored

In [44]:
# 가치분석 크롤링 주소 세팅
value_url_1 = "https://navercomp.wisereport.co.kr/v2/company/cF4002.aspx?cmp_cd="
value_url_2 = "&frq=0&rpt=5&finGubun=MAIN&frqTyp=0&cn=&encparam="
value_url_all = value_url_1 + code + value_url_2 + encparam
print(value_url_all)

# 크롤링
json_string = requests.get(value_url_all).text

#가치분석은 json형태로 
value_list = json.loads(json_string)

# 년도 데이터 추출
colnm_list = value_list['YYMM']
colnm_list_new = ['type']
for col_nm in colnm_list[:5]:
    colnm_list_new.append( col_nm[0:7].replace("/","."))
print(colnm_list_new)

# 내용 데이터 필요 row Index 추출
data3 = DataFrame(value_list['DATA'])
acc_idx = [0,4,8,12,16,19,22,25,28,31,34,35]
data4 = data3.iloc[acc_idx, :]

# 내용 데이터 필요 column Index  추출
col_idx = [1,3,4,5,6,7]


https://navercomp.wisereport.co.kr/v2/company/cF4002.aspx?cmp_cd=005930&frq=0&rpt=5&finGubun=MAIN&frqTyp=0&cn=&encparam=QVVXbVlPbzd5R0M5c2JKNUcwUzFrZz09
['type', '2014.12', '2015.12', '2016.12', '2017.12', '2018.12']


## 투자데이터 가져오기

In [45]:
value_data  = make_table(data4,col_idx, colnm_list_new,code )
value_data

type,DATE,code,EPS,BPS,CPS,SPS,PER,PBR,PCR,PSR,EV/EBITDA,DPS,현금배당수익률,현금배당성향(%)
0,2014.12,005930,2713.47,21664.1,4346.65,24240.6,9.781,1.22507,6.106,1.095,3.77368,400,1.50716,12.9967
1,2015.12,005930,2197.65,23714.8,4709.47,23587.9,11.467,1.06263,5.351,1.0685,3.30862,420,1.66667,16.4151
2,2016.12,005930,2735.21,26635.6,5782.1,24632.2,13.1765,1.35308,6.233,1.463,4.15758,570,1.58158,17.8085
3,2017.12,005930,5421.23,30427,8150.88,31413.9,9.4,1.67483,6.252,1.622,3.91336,850,1.66798,14.0921
4,2018.12,005930,6024.16,35342.4,9200.33,33458.4,6.42413,1.095,4.20637,1.15666,1.98658,1416,3.65892,21.9163
